In [1]:
import json
import requests
import pandas as pd
import os

   


In [ ]:
api_key = os.getenv("rwa_xyz_api")

def fetch_top_rwa_assets(api_key, limit=1000):
    """
    Fetch top RWA assets sorted by market value
    """
    all_results = []
    page = 1
    per_page = 100
    
    while len(all_results) < limit:
        print(f"📄 Page {page}...", end=" ")
        
        # Sort by market value (descending) - this gets top assets first
        myQuery = json.dumps({
            "sort": {
                "direction": "desc",
                "field": "market_value_dollar"  # API handles the nested .val internally
            },
            "pagination": {
                "page": page,
                "perPage": per_page
            }
        })
        
        response = requests.get(
            "https://api.rwa.xyz/v4/assets",
            params={"query": myQuery},
            headers={"Authorization": f"Bearer {api_key}"}
        )
        
        if response.status_code != 200:
            print(f"\n❌ Error: {response.text}")
            break
        
        data = response.json()
        results = data.get("results", [])
        
        all_results.extend(results)
        print(f"✓ ({len(all_results)} total)")
        
        if not results or len(results) < per_page:
            break
        
        page += 1
    
    return all_results[:limit]

# Fetch top 1000
print("🔄 Fetching top 1000 RWA assets...\n")
top_assets = fetch_top_rwa_assets(api_key, limit=1000)

print(f"\n✅ Fetched {len(top_assets)} assets!")

# Convert and select columns
df = pd.json_normalize(top_assets)

# Filter out assets with no market value
df = df[df['market_value_dollar.val'] > 0].copy()

print(f"📊 Assets with market value: {len(df)}")

# Select useful columns
key_columns = [
    'name', 'ticker', 'underlying_asset_classes',
    'market_value_dollar.val',
    'daily_transfer_volume_token',
    'daily_active_addresses_count',
    'token_count',
    'top_holder_concentration',
    'trailing_7_day_transfer_count',
    'trailing_30_day_transfer_count',
]

existing = [col for col in key_columns if col in df.columns]
df_clean = df[existing].copy()

print(f"\n✅ Final dataset: {len(df_clean)} rows × {len(df_clean.columns)} columns")
print(f"\n🔍 Preview:")
print(df_clean.head(10))

print(f"\n💰 Market value range:")
print(df_clean['market_value_dollar.val'].describe())

🔄 Fetching top 1000 RWA assets...

📄 Page 1... 
❌ Error: {"message":"Invalid API key. Please ensure that the API key is correct and try again."}

✅ Fetched 0 assets!


KeyError: 'market_value_dollar'